In [12]:
#from tkinter import Tk, ttk, StringVar, Label, Entry, Button, Text, filedialog, END, W, E, N
from tkinter import *
from tkinter import Tk, ttk, StringVar, Label, Entry, Button, Text, filedialog, END, W, E, N
import cv2
import os
from PIL import ImageTk, Image
import tensorflow as tf
import math
import numpy as np

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util

#from object_detection.utils import visualization_utils as vis_util
    

################################################################################################
# Parameters
################################################################################################
MODEL_NAME = os.getcwd()
LABEL_PATH = os.path.join(MODEL_NAME, 'data')
PATH_TO_CKPT = os.path.join(MODEL_NAME, 'frozen_inference_graph.pb')
PATH_TO_LABELS = os.path.join(LABEL_PATH, 'surgery_label_displayname_map.pbtxt')

NUM_CLASSES = 15
MIN_CONFIDENCE_LEVEL = 0.7


################################################################################################
# Global Variables
################################################################################################
detection_graph = None   # to store detection model
category_names = None    # to store class name mapping
category_descs = None     # to store class desc. (display name) mapping

    
def cropping_new_images(test_images_paths, photo_num, coordinates , width, height):
    image = cv2.imread(test_images_paths)
#     ymin = math.ceil(coordinates[0] * ratio[0])
#     xmin = math.ceil(coordinates[1] * ratio[1])
#     ymax = math.ceil(coordinates[2] * ratio[0])
#     xmax = math.ceil(coordinates[3] * ratio[1])
#     ymin = ymin - 5
#     xmin = xmin - 5
#     xmax = xmax + 5
#     ymax = ymax + 5
    ymin = coordinates[0]
    xmin = coordinates[1]
    ymax = coordinates[2] 
    xmax = coordinates[3]
    _, tail = os.path.split(test_images_paths)
#     cropped_image = tf.image.crop_to_bounding_box(image, ymin, xmin, ymax - ymin, xmax - xmin)
#     output_image = tf.image.encode_png(cropped_image)
    output_image=image[ymin:ymax, xmin:xmax]
    file_name = os.path.join('surgery_data', tail.replace('.jpg', '_' + 'new_' + str(photo_num) + '.jpg'))
    cv2.imwrite(file_name, cv2.resize(output_image,(width, height)))


################################################################################################
# align the detected image with original image
################################################################################################
def alignImage(original_image_path, detect_image_path):
    MIN_MATCH_COUNT = 4

    imgname2 = original_image_path
    imgname1 = detect_image_path

    img1 = cv2.imread(imgname1)
    img2 = cv2.imread(imgname2)
    h, w, _ = img1.shape
    h1, w1, _ = img2.shape
    if h1 >= h:
        h = h1
    if w1 > w:
        w = w1
    height = (math.floor(h / 10) + 1) * 10
    width = (math.floor(w / 10) + 1) * 10
    img1 = cv2.resize(img1, (width, height))
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.xfeatures2d.SIFT_create()

    # # (3) Create flann matcher
    matcher = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), {})

    # # (4) Detect keypoints and compute keypointer descriptors
    kpts1, descs1 = sift.detectAndCompute(gray1, None)
    kpts2, descs2 = sift.detectAndCompute(gray2, None)

    # # (5) knnMatch to get Top2
    matches = matcher.knnMatch(descs1, descs2, 2)
    # Sort by their distance.
    matches = sorted(matches, key=lambda x:x[0].distance)

    # # (6) Ratio test, to get good matches.
    good = [m1 for (m1, m2) in matches if m1.distance < 0.7 * m2.distance]

    canvas = img2.copy()

    # # (7) find homography matrix
    if len(good) > MIN_MATCH_COUNT:
        # # (queryIndex for the small object, trainIndex for the scene )
        src_pts = np.float32([ kpts1[m.queryIdx].pt for m in good ]).reshape(-1, 1, 2)
        dst_pts = np.float32([ kpts2[m.trainIdx].pt for m in good ]).reshape(-1, 1, 2)
        # # find homography matrix in cv2.RANSAC using good match points
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        # matchesMask2 = mask.ravel().tolist()
        h, w = img1.shape[:2]
        pts = np.float32([ [0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0] ]).reshape(-1, 1, 2)
        dst = cv2.perspectiveTransform(pts, M)
        # # 绘制边框
        cv2.polylines(canvas, [np.int32(dst)], True, (0, 255, 0), 3, cv2.LINE_AA)
    else:
        print("Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT))

    # # (8) drawMatches
    matched = cv2.drawMatches(img1, kpts1, canvas, kpts2, good, None)  # ,**draw_params)

    # # (9) Crop the matched region from scene
    h, w = img1.shape[:2]
    pts = np.float32([ [0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0] ]).reshape(-1, 1, 2)
    dst = cv2.perspectiveTransform(pts, M)
    perspectiveM = cv2.getPerspectiveTransform(np.float32(dst), pts)
    found = cv2.warpPerspective(img2, perspectiveM, (w, h))
    return found


def difference_detection_program(original_images_path, after_surgery_img2_path, photo_number_name, final_file_location=[]):
    
    found = alignImage(original_images_path, after_surgery_img2_path)
    height, width = found.shape[:2]
    newW = width + 10
    newH = height + 10 
    found = cv2.resize(found, (newW, newH))
    im1 = cv2.imread(after_surgery_img2_path)
    im = cv2.resize(found, (width, height))
    im1 = cv2.resize(im1, (width, height))
    diff = cv2.subtract(im, im1)
    im = cv2.resize(diff, (width, height))
    imgray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(imgray, 120, 180, 0)
    _, contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(im1, contours, -1, (127, 255, 0), 4)
    final_file_names = 'final_result_' +  str(photo_number_name) + ".jpg"
    
    cv2.imwrite(final_file_names, im1)
    final_file_location.append(final_file_names) 


def highlighting_brown_color(panel3,images_path):
    image123 = cv2.imread(images_path,cv2.IMREAD_UNCHANGED)
    #inRange(hsv, Scalar(0, 100, 20), Scalar(10, 255, 255), mask1);
    #inRange(hsv, Scalar(170, 100, 20), Scalar(180, 255, 255), mask2);
    red = np.array([0, 140, 20])
    red1 = np.array([10, 255, 255])
    red2 = np.array([170, 100, 20])
    red3 = np.array([180, 250, 250])
    #red4 = np.array([3, 100, 20])
    #red5 = np.array([20, 255, 200])
    hsv = cv2.cvtColor(image123, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv, red, red1)
    mask2 = cv2.inRange(hsv, red2, red3)
    #mask3 = cv2.inRange(hsv, red4, red5)
    mask = mask1 + mask2 #+ mask3
    ret, thresh = cv2.threshold(mask, 0, 255, 0)
    _, contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image123, contours, -1, (127, 255, 0), 1)
    
    filename = 'final.jpg'
    cv2.drawContours(image123, contours, -1, (127, 255, 0), 2)
    cv2.imwrite(filename, image123)
    img = Image.open(filename)
    img = img.resize((400,400))
    # resize the final image to be displayed
    #ratio_h = panel_h / maxH
    #ratio_w = panel_w / total_width
    #ratio_min = min(ratio_h, ratio_w)
    #display_h = int(maxH*ratio_min)
    #display_w = int(total_width*ratio_min)
    #img = cv2.resize(img,(400,600))
    img_resized = ImageTk.PhotoImage(img)
    panel3.image = img_resized
    panel3.configure(image = img_resized) 
    
    
    
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)  

def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
                real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})
            
            output_dict['num_detections'] = int(output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict     


################################################################################################
# Load Object Detection Model
################################################################################################
def load_model(modelFilePath):
   
    # load model
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(modelFilePath, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return detection_graph

  
################################################################################################
# Load Object Detection Label
################################################################################################  
def load_label(label_file_path, num_classes, use_display_name=True):
    label_map = label_map_util.load_labelmap(label_file_path)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes, use_display_name=use_display_name)
    category_index = label_map_util.create_category_index(categories)  
    return category_index;
    
    
def detection_surgery(test_images_paths, photo_num, result_output1, file_path_to_extarcted_one=[], file_path_to_after=[], final_file_location=[]):
    global detection_graph
    global category_names
    global category_descs

    image = Image.open(test_images_paths)
    image_np = load_image_into_numpy_array(image)
#     image_np_expanded = np.expand_dims(image_np, axis=0)
    output_dict = run_inference_for_single_image(image_np, detection_graph)

    head, tail = os.path.split(test_images_paths)
    boxes = output_dict['detection_boxes']
    scores = output_dict['detection_scores']
    classes = output_dict['detection_classes']
    boxes_shape = boxes.shape
    im_width, im_height = image.size
    frequency = [0] * NUM_CLASSES
    tot = 0 # number of class detected
    
    eq_list = [-1] * 100
    
    # Detection Part
    for i in range(boxes_shape[0]):
        if (scores[i] > MIN_CONFIDENCE_LEVEL):
            ymin = int(boxes[i, 0] * im_height)
            xmin = int(boxes[i, 1] * im_width)
            ymax = int(boxes[i, 2] * im_height)
            xmax = int(boxes[i, 3] * im_width)
            coordinate_of_object = []
            #ratio_to_the_original = []
            # +/ - of y and x max/min value here is to give extra space in the image in order to avoid cases which some part of the object were cropped out
            coordinate_of_object.append(ymin - 10)
            coordinate_of_object.append(xmin - 10)
            coordinate_of_object.append(ymax)
            coordinate_of_object.append(xmax + 10)
            #file_path_to_original = os.path.join(os.getcwd(), 'original', classes[i])
            file_name_original = os.path.join('original', str(output_dict['detection_classes'][i]) + '.jpg')
            im = Image.open(file_name_original)
            w, h = im.size
#             ratio_to_the_original.append(float(h / im_height))
#             ratio_to_the_original.append(float(w / im_width))
            cropping_new_images(test_images_paths, classes[i], coordinate_of_object, w, h)
            eq_list[tot] = classes[i]  # store class id 
            tot += 1
            frequency[i] += 1
            file_name = os.path.join('surgery_data', tail.replace('.jpg', '_' + 'new_' + str(classes[i]) + '.jpg'))
            
#             if photo_num % 2 != 0:
#                 file_name = os.path.join(os.getcwd(), file_name)
#                 file_path_to_extarcted_one.append(file_name)  # getting original image that used for comparison
            if photo_num % 2 == 0:
                file_name = os.path.join(os.getcwd(), file_name)
                file_path_to_after.append(file_name) 
                file_path_to_extarcted_one.append(file_name_original)
            
#     boxes = output_dict['detection_boxes']
#     scores = output_dict['detection_scores']
#     boxes_shape = boxes.shape
    tot = 0 
    class_name = ''
    model_numbers = []
    quantity_of_models = []
    description_of_models = []
    des_model_num = ''
    
    # loop for each class and append the result if count > 0
    for i in range(1, NUM_CLASSES+1):
        
        if eq_list.count(i) > 0:
            
            class_name = category_names[i]['name']
            des_model_num = category_descs[i]['name']
            model_numbers.append(class_name)
            quantity_of_models.append(str(eq_list.count(i)))
            description_of_models.append(des_model_num)
#     if photo_num % 2 != 0:
#         result_output.insert(END, 'Detection Result from before of the Surgery ' + '\n' + '\n')
#         result_output.insert(END, 'Model Number  ' + 'Description                      ' + 'Qty' + '\n')
    if photo_num % 2 == 0:
        #result_output1.insert(END, 'Detection Result ' + '\n' )
        result_output1.delete(1.0, END)
        result_output1.insert(END, 'Qty   ' + ' Model ' + '\n')
        result_output1.insert(END, '------' + ' ---------------------------------------------------------------' + '\n')
    for i, j, k in zip(model_numbers, description_of_models, quantity_of_models):
        if photo_num % 2 == 0:
            result_output1.insert(END, k.ljust(6) + ' ' + i + ' (' + j + ')\n')
#         else:
#             result_output.insert(END, i + '  ' + j + '   ' + k + '\n')
    '''
    if photo_num % 2 == 0:
        tot_photo_count = 0

        for x, y in zip(file_path_to_extarcted_one, file_path_to_after):
            #difference_detection_program(x, y, tot_photo_count, final_file_location)
            highlighting_brown_color(x,y, tot_photo_count, final_file_location)
            tot_photo_count += 1
    '''
    
#########################################################################    
# function to combine image list to 1 image
#########################################################################        
def combine_all_together(panel3, final_image_list=[]):
    loaded_images = []
    # panel size
    panel_h = panel3.winfo_height()
    panel_w = panel3.winfo_width()
   
    # to calculate the final image size
    maxH = 0
    total_width = 0
    for x in final_image_list:
        img = cv2.imread(x)
        h, w = img.shape[:2]
        maxH = max(h, maxH)
        total_width += w
        
    # combine all images to final image    
    for x in final_image_list:
        img = cv2.imread(x)
        h, w = img.shape[:2]
        data = np.zeros((maxH,w,3), dtype= np.uint8)
       
        data[0:h, 0:w] = img
        loaded_images.append(data)
    numpy_horizontal = np.hstack(loaded_images)
    filename = 'final.jpg'
    cv2.imwrite(filename, numpy_horizontal)
    img = Image.open(filename)
    
    # resize the final image to be displayed
    ratio_h = panel_h / maxH
    ratio_w = panel_w / total_width
    ratio_min = min(ratio_h, ratio_w)
    display_h = int(maxH*ratio_min)
    display_w = int(total_width*ratio_min)
    img = img.resize((400, 400))
    
    img_resized = ImageTk.PhotoImage(img)
    panel3.image = img_resized 
    panel3.configure(image=img_resized) 
 

#########################################################################    
# function to browse and display image
#########################################################################    
def browsefunc(filepath, panel,filetypes=None):
    
    filename =  filedialog.askopenfilename(filetypes=filetypes)
    filepath.set(filename)
    if (filename):
        img = Image.open(filename)
        h = panel.winfo_height() 
        w = panel.winfo_width() 
        img = img.resize((h, w)) 
        img_resized = ImageTk.PhotoImage(img)
        panel.image = img_resized 
        panel.configure(image=img_resized)
    
    

if __name__ == '__main__':
    
    # initial load model and label
#     global detection_graph
    detection_graph = load_model(PATH_TO_CKPT)
#     global category_names
    category_names = load_label(PATH_TO_LABELS, NUM_CLASSES, use_display_name=False)
#     global category_descs
    category_descs = load_label(PATH_TO_LABELS, NUM_CLASSES)  
    
    root = Tk()  
    root.title('Surgery Equipment Detection')
#     folder_path1 = StringVar()
    folder_path2 = StringVar()
#     label1 = Label(root, text="Before the Surgery").grid(row=0, column=0, sticky=W)
#     entry1 = Entry(root, width=70, textvariable=folder_path1).grid(row=1, column=0, pady=10)
#     browsebutton1 = Button(root, text="Browse", command=lambda:folder_path1.set(filedialog.askopenfilename()))
#     browsebutton1.grid(row=0, column=0)
#     file = os.path.join(os.getcwd(), 'logo_images', 'b.jpg')
#     img1 = ImageTk.PhotoImage(Image.open(file))
#     panel1 = Label(root, height="400", width="400", image=img1)
#     panel1.grid(row=2, column=0)
#     loadbutton1 = Button(root, text="Load Image", command=lambda:browsefunc(folder_path1.get(), panel1))
#     loadbutton1.grid(row=0, column=0, sticky=E)
    
   
    # Create a container to hold output message
    detectFrame = ttk.LabelFrame(root, text='  1. Object Detection ')
    Label(detectFrame, text="Select Image file for detection").grid(row=0, column=1, sticky=W, padx=10)
    entry2 = Entry(detectFrame, width=70, textvariable=folder_path2).grid(row=1, column=1, padx=5 , pady=10)
    browsebutton2 = Button(detectFrame, text="Browse", command=lambda:browsefunc(folder_path2, panel2,  filetypes=[("JPG Files", "*.jpg")]))
    browsebutton2.grid(row=1, column=1, padx=5 , pady=10, sticky=E)
    
    file2 = os.path.join(os.getcwd(), 'logo_images', 'a.jpg')
    img2 = ImageTk.PhotoImage(Image.open(file2))
    panel2 = Label(detectFrame, height="400", width="400", image=img2)
    panel2.grid(row=2, column=1)
#     loadbutton2 = Button(root, text="Load Image", command=lambda:browsefunc(folder_path2.get(),panel2))
#     loadbutton2.grid(row=0, column=1, sticky=E)
    detectFrame.grid(column=1, row=1, pady=5, padx=5)
    
    
    diffFrame = ttk.LabelFrame(root, text=' 2. Check Difference ')
    file3 = os.path.join(os.getcwd(), 'logo_images', 'f.jpg')
    img3 = ImageTk.PhotoImage(Image.open(file3))
    panel3 = Label(diffFrame, height="400", width="600", image=img3)
    panel3.grid(row=2, column=3, pady=10)
    diffFrame.grid(column=2, row=1, pady=50, padx=5, sticky=N)
    
    file_path_to_before_surgery = []
    file_path_to_after_surgery = []
    final_file_locations = []
    
#     result_output = Text(root, width=50, height=5)
#     result_output.grid(row=5, column=0)
    scrollbar = Scrollbar(detectFrame)
    scrollbar.grid(row = 5, column = 2, sticky=E)
    result_output1 = Text(detectFrame, width=70, height=5,yscrollcommand=scrollbar.set)
    result_output1.grid(row=5, column=1, padx=5 , pady=5)
    scrollbar.config(command=result_output1.yview)
#     detectButton1 = Button(root, text="DETECT", command=lambda:detection_surgery(folder_path1.get(), 3, file_path_to_before_surgery, None, final_file_locations)).grid(row=3, column=0)
    detectButton2 = Button(detectFrame, text="Detect", command=lambda:detection_surgery(folder_path2.get(), 2, result_output1,file_path_to_before_surgery, file_path_to_after_surgery, final_file_locations), width = 15, bg = "powder blue").grid(row=3, column=1)

    #defeatCheckButton = Button(diffFrame, text="Check for Difference", command=lambda:combine_all_together(panel3, final_file_locations), bg = "powder blue").grid(row=3, column=3, padx=5 , pady=5)
    
    defeatCheckButton = Button(diffFrame, text="Check for Difference", command=lambda:highlighting_brown_color(panel3, folder_path2.get()), bg = "powder blue").grid(row=3, column=3, padx=5 , pady=5)
    root.mainloop()

